# Noise2Noise

CARE network you trained in part 1 require that you acquire additional data for the purpose of denoising. For CARE we used a paired acquisition with high SNR, for Noise2Noise we had paired noisy acquisitions.

This notebook uses a set of images with different levels of noise from SEM dataset.

We'll use the [Careamics](https://careamics.github.io) library

## Reference

Jaakko Lehtinen, Jacob Munkberg, Jon Hasselgren, Samuli Laine, Tero Karras, Miika Aittala, Timo Aila 
[Noise2Noise: Learning Image Restoration without Clean Data](https://arxiv.org/abs/1803.04189)




### Mandatory actions
<div class="alert alert-danger">
Set your python kernel to <code>regression</code> <br>
</div>



<div class="alert alert-block alert-info"><h3>Task 2.2: Bla</h3>
    
This notebook uses a single image from the SEM data from the Noise2Noise notebook.

If you brought your own raw data, use that instead! The only requirement is that the noise in your data is pixel-independent and zero-mean. If you're unsure whether your data fulfills that requirement or you don't yet understand why it is necessary ask one of us to discuss!

If you don't have suitable data of your own, feel free to find some online or ask your fellow course participants. You can however also stick with the SEM data provided here and compare the results to what you achieved with Noise2Noise in the previous part.


</div>

<hr style="height:2px;"><div class="alert alert-block alert-success"><h1>Checkpoint 1</h1>

</div>

In [ ]:
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import tifffile
from careamics_portfolio import PortfolioManager

from careamics import CAREamist
from careamics.config import create_n2n_configuration

### Part 1. Prepare the data

SEM dataset consists of EM images with 7 different levels on noise

In [ ]:
# Download files
portfolio = PortfolioManager()
root_path = Path("./data")
files = portfolio.denoising.N2N_SEM.download(root_path)

### Visualize training data

In this cell we can see the different levels of noise in the SEM dataset

In [ ]:
# Load images
train_image = tifffile.imread(root_path / "denoising-N2N_SEM.unzip/SEM/train.tif")
print(f"Train image shape: {train_image.shape}")
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(train_image[0], cmap="gray")
ax[0].set_title("Train image highest noise level")
ax[1].imshow(train_image[-1], cmap="gray")
ax[1].set_title("Train image lowest noise level")

#### Zoom
The noise level is hard to see at this zoom level. Let's also look at a smaller crop of them! Play around with this until you have a feeling for what the data looks like.

In [ ]:
train_image = tifffile.imread(files[0])
print(f"Train image shape: {train_image.shape}")
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(train_image[0,100:356, 500:756], cmap="gray")
ax[0].set_title("Train image highest noise level")
ax[1].imshow(train_image[-1, 100:356, 500:756], cmap="gray")
ax[1].set_title("Train image lowest noise level")

### Part 2. Create configuraion

In [ ]:
training_config = create_n2n_configuration(
    experiment_name="LevitatingFrog",
    data_type="array",
    axes="YX",
    patch_size=[128, 128],
    batch_size=128,
    num_epochs=30,
    logger="tensorboard"

)

In [ ]:
# Visualize training configuration
print(training_config)

#### Initialize the Model

Create a Pytorch Lightning module

Please take as look at the [documentation](https://careamics.github.io) to see the full list of parameters and configuration options

In [ ]:
engine = CAREamist(source=training_config)


### Part 3. Run training 

We need to specify the paths to training and validation data

In [ ]:
engine.train(train_source=train_image[0],
            train_target=train_image[4],
            val_source=train_image[0],
            val_target=train_image[4]
)

### Part 4. Prediction

### Visualize test data


In [ ]:
# Load images
test_image = tifffile.imread(root_path / "denoising-N2N_SEM.unzip/SEM/test.tif")
print(f"Test image shape: {train_image.shape}")
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(test_image[0], cmap="gray")
ax[0].set_title("Test image highest noise level")
ax[1].imshow(test_image[-1], cmap="gray")
ax[1].set_title("Test image lowest noise level")

In [ ]:
prediction = engine.predict(source=test_image[4], tile_size=(256, 256))

### Visualize predictions

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(test_image[-1], cmap="gray")
ax[0].set_title("Test image lowest noise level")
ax[1].imshow(prediction, cmap="gray")
ax[1].set_title("Test image highest noise level")